## Imports and Libraries

In [1]:
# Import of libraries 
import sys  # imports sys package for sue below
from pylab import *  # contains some useful stuff for plotting
import datetime
import time  # imports a package that allows us to see how long processes take
import math  # imports math package
import numpy as np # imports numpu package as np for short
import sympy as sym    # we import the sympy package
import scipy
from scipy.special import spherical_jn, spherical_yn
from scipy import interpolate
from scipy.interpolate import UnivariateSpline 
from scipy.integrate import cumulative_trapezoid
from scipy.integrate import solve_ivp
from scipy.optimize    import root_scalar
from scipy.interpolate import interp1d
from scipy.integrate import quad, simpson

import matplotlib
from matplotlib import pyplot as plt   # import package for plotting
import matplotlib.dates as mdates
from matplotlib.transforms import Transform
from matplotlib.ticker import (AutoLocator, AutoMinorLocator)



## Defining the dynamical equations ## 

In [2]:
# definition of ODE systems etc.
  
def fmtk(x):
    return '{:.3f}'.format(x)

# Background ODE system in conformal time and comoving synchronous gauge, as well as rescaled quantities eta/eta_i, 
# rho(m/r)*eta_i/MPL and a/a_i
def background_odes(eta, y):
    rho_m, rho_r, a, GammaT = y
    # Equations as function of eta
    da = a**2 * np.sqrt((rho_m + rho_r)/(3*M_Pl**2))
    drho_m = -3 * (da/a) * rho_m - GammaT * rho_m
    drho_r = -4 * (da/a) * rho_r + GammaT * rho_m
    dGammaT = (da/a) * GammaT + 3 * GammaT**2
    return [drho_m, drho_r, da, dGammaT]

# Event to stop integration when rho_m is much smaller than rho_r
def stop_event(eta, y):
    rho_m, rho_r, a, GammaT = y
    return rho_m / rho_r - 1e-10
stop_event.terminal = True
stop_event.direction = -1

# Event to stop integration when rho_m is much smaller than rho_r
def stop_event_pre(eta, y):
    a, gamma, rho_m, rho_r, GammaT, delta_m, delta_r, theta_r, epsilon = y
    return rho_m / rho_r - 1e-10
stop_event.terminal = True
stop_event.direction = -1


# The total (back + 1st-order pert) ODE system pre-evaporation to avoid interpolated functions
def ODE_pre(eta, y):
    a, gamma, rho_m, rho_r, GammaT, delta_m, delta_r, theta_r, epsilon = y

    term = 2 * k**2 * epsilon + a**2 * (rho_m * delta_m + rho_r * delta_r) / M_Pl**2
    
    da = a**2 * np.sqrt((rho_m + rho_r)/(3*M_Pl**2))
    dgamma = (np.sqrt(3* M_Pl**2 /(rho_m+rho_r))/a) * term
    drho_m = -3 * (da/a) * rho_m - GammaT * rho_m
    drho_r = -4 * (da/a) * rho_r + GammaT * rho_m
    dGammaT = (da/a) * GammaT + 3 * GammaT**2
    ddelta_m = -dgamma/2
    ddelta_r = - (4/3) * (theta_r + dgamma/2) + (GammaT * rho_m/rho_r) * (delta_m - delta_r)
    dtheta_r = (k**2 / 4) * delta_r - (GammaT * rho_m / rho_r) * theta_r
    depsilon = (2 * a**2 * rho_r  * theta_r) / (3 * M_Pl**2 * k**2)

    return[da, dgamma, drho_m, drho_r, dGammaT, ddelta_m, ddelta_r, dtheta_r, depsilon]

# The total (back + 1st-order pert) ODE system post-evaporation to avoid interpolated functions
def ODE_pdefost(eta, y):
    a, gamma, rho_m, rho_r, delta_m, delta_r, theta_r, epsilon = y

    term = 2 * k**2 * epsilon + a**2 * (rho_m * delta_m + rho_r * delta_r) / M_Pl**2  # There was a typo here! Missing M_Pl**2
    
    da = a**2 * np.sqrt((rho_m + rho_r)/(3*M_Pl**2))
    dgamma = (np.sqrt(3* M_Pl**2 /(rho_m+rho_r))/a) * term
    drho_m = -3 * (da/a) * rho_m
    drho_r = -4 * (da/a) * rho_r
    ddelta_m = -dgamma/2
    ddelta_r = - (4/3) * (theta_r + dgamma/2)
    dtheta_r = (k**2 / 4) * delta_r
    depsilon = (2 * a**2 * rho_r  * theta_r) / (3 * M_Pl**2 * k**2)

    return[da, dgamma, drho_m, drho_r, ddelta_m, ddelta_r, dtheta_r, depsilon]



In [19]:
# WE DEFINE THE JACOBIAN TO PASS TO THE SOLVE_IVP FUNCTION AND INCREASE ACCURACY

def jac_pre(eta,y):
    a, gamma, rho_m, rho_r, GammaT, delta_m, delta_r, theta_r, epsilon = y
    rho_tot= rho_m + rho_r
    #M_Pl = 1.0

    J = np.zeros((9,9))
    J[0,0]= 2*a*np.sqrt(rho_tot/(3*M_Pl**2)) # da'/da
    J[0,1]= 0 # da'/dgamma
    J[0,2]= a**2/(2*np.sqrt(3*rho_tot*M_Pl**2)) # da'/drho_m
    J[0,3]= a**2/(2*np.sqrt(3*rho_tot*M_Pl**2)) # da'/drho_r
    J[0,4]= 0 # da'/dGammaT
    J[0,5]= 0 # da'/ddelta_m
    J[0,6]= 0 # da'/ddelta_r
    J[0,7]= 0 # da'/dtheta_r
    J[0,8]= 0 # da'/depsilon
    J[1,0]= - (2*np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) + np.sqrt(3)*(rho_m*delta_m+rho_r*delta_r)/(M_Pl*np.sqrt(rho_tot)) #dgamma'/da'
    J[1,1]= 0 # dgamma'/dgamma
    J[1,2]= - (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a*rho_tot**(3/2)) + a*np.sqrt(3)*(delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*M_Pl*(rho_tot)**(3/2))
    J[1,3]= - (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a*rho_tot**(3/2)) + a*np.sqrt(3)*(delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*M_Pl*(rho_tot)**(3/2))
    J[1,4]= 0 # dgamma'/dGammaT
    J[1,5]= a*np.sqrt(3/rho_tot)*rho_m/M_Pl # dgamma'/ddelta_m
    J[1,6]= a*np.sqrt(3/rho_tot)*rho_r/M_Pl # dgamma'/ddelta_r
    J[1,7]= 0 # dgamma'/theta_r
    J[1,8]= ((2*k**2)/a)*np.sqrt((3*M_Pl**2)/rho_tot) # dgamma'/depsilon
    J[2,0]= -3*rho_m*np.sqrt(rho_tot/(3*M_Pl**2)) # drho_m'/da
    J[2,1]= 0 # drho_m'/ dgamma
    J[2,2]= -a*np.sqrt(3*rho_tot/M_Pl**2) - GammaT - (3*a*rho_m)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # drho_m'/drho_m
    J[2,3]= -(3*a*rho_m)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # drho_m'/drho_r
    J[2,4]= -rho_m # drho_m'/dGammaT
    J[2,5]= 0 
    J[2,6]= 0
    J[2,7]= 0
    J[2,8]= 0 # drho_m'/depsilon
    J[3,0]= -4*rho_r*np.sqrt(rho_tot/(3*M_Pl**2)) # drho_r'/da
    J[3,1]= 0 # drho_r'/ dgamma
    J[3,2]= - (2*a*rho_r)/(np.sqrt(3*M_Pl**2*rho_tot)) + GammaT # drho_r'/drho_m 
    J[3,3]= -4*a*np.sqrt(rho_tot/(3*M_Pl**2)) - (2*a*rho_r)/(np.sqrt(3**M_Pl**2*rho_tot)) # drho_r'/drho_r
    J[3,4]= rho_m # drho_r'/dGammaT
    J[3,5]= 0
    J[3,6]= 0
    J[3,7]= 0
    J[3,8]= 0
    J[4,0]= np.sqrt(rho_tot/(3*M_Pl**2))*GammaT # dGammaT'/da
    J[4,1]= 0
    J[4,2]= (a*GammaT)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # dGammaT'/drho_m
    J[4,3]= (a*GammaT)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # dGammaT'/drho_r
    J[4,4]= a*np.sqrt(rho_tot/(3*M_Pl**2)) + 6*GammaT # dGammaT'/dGammaT
    J[4,5]= 0
    J[4,6]= 0
    J[4,7]= 0
    J[4,8]= 0
    J[5,0]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) - (np.sqrt(3*M_Pl**2)/(2*np.sqrt(rho_tot)))*(rho_m*delta_m+rho_r*delta_r) # ddelta_m'/da
    J[5,1]= 0 # ddelta_m'/dgamma 
    J[5,2]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) - ((a*np.sqrt(3))/(2*M_Pl))*((delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*rho_tot**(3/2))) # ddelta_m'/drho_m
    J[5,3]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) - ((a*np.sqrt(3))/(2*M_Pl))*((delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*rho_tot**(3/2)))# ddelta_m'/drho_r
    J[5,4]= 0 # ddelta_m'/dGammaT
    J[5,5]= - a/(2*M_Pl)*np.sqrt(3/rho_tot)*rho_m # ddelta_m'/ddelta_m
    J[5,6]= - a/(2*M_Pl)*np.sqrt(3/rho_tot)*rho_r # ddelta_m'/ddelta_r
    J[5,7]= 0
    J[5,8]= -k**2/a*np.sqrt((3*M_Pl**2)/rho_tot) # ddelta_m'/depsilon
    J[6,0]= 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) - 4/3*np.sqrt(3)/(2*M_Pl*np.sqrt(rho_tot))*(rho_m*delta_m+rho_r*delta_r) # ddelta_r'/da
    J[6,1]= 0 # ddelta_m'/dgamma
    J[6,2]= - 4/3*((a*np.sqrt(3))/(2*M_Pl))*((delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*rho_tot**(3/2))) + 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) + (GammaT/rho_r)*(delta_m-delta_r) # ddelta_r'/drho_m
    J[6,3]= - 4/3*((a*np.sqrt(3))/(2*M_Pl))*((delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*rho_tot**(3/2))) + 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) - ((GammaT*rho_m)/rho_r**2)*(delta_m-delta_r) # ddelta_r'/drho_r
    J[6,4]= (rho_m/rho_r)*(delta_m-delta_r) # # ddelta_r'/dGammaT
    J[6,5]= GammaT*rho_m/rho_r - (2*a*rho_m)/(M_Pl*np.sqrt(3*rho_tot)) # ddelta_r'/ddelta_m
    J[6,6]= -GammaT*rho_m/rho_r - (2*a*rho_r)/(M_Pl*np.sqrt(3*rho_tot)) # ddelta_m'/ddelta_r
    J[6,7]= -4/3 # ddelta_m'/dtheta_r
    J[6,8]= -4/3*(k**2/a)*np.sqrt((3*M_Pl**2)/rho_tot) # ddelta_m'/depsilon
    J[7,0]= 0 # dtheta_r'/da
    J[7,1]= 0 # dtheta_r'/dgamma
    J[7,2]= -(GammaT/rho_r)*theta_r # dtheta_r'/drho_m
    J[7,3]= ((GammaT*rho_m)/(rho_r**2))*theta_r # dtheta_r'/drho_r
    J[7,4]= - (rho_m/rho_r)*theta_r # dtheta_r'/dGammaT
    J[7,5]= 0 # dtheta_r'/ddelta_m
    J[7,6]= k**2/4 # dtheta_r'/ddelta_r
    J[7,7]= -GammaT*(rho_m/rho_r) # dtheta_r'/dtheta_r
    J[7,8]= 0 # dtheta_r'/depsilon
    J[8,0]= (4*a*rho_r*theta_r)/(3*k**2) # depsilon'/da
    J[8,1]= 0 # depsilon'/dgamma
    J[8,2]= 0 # depsilon'/drho_m
    J[8,3]= (2*a**2*theta_r)/(3*k**2) # depsilon'/drho_r
    J[8,4]= 0 # depsilon'/dGammaT
    J[8,5]= 0 # depsilon'/ddelta_m
    J[8,6]= 0 # depsilon'/ddelta_r
    J[8,7]= (2*a**2*rho_r)/(3*k**2) # depsilon'/dtheta_r 
    J[8,8]= 0 # depsilon'/depsilon

    return J

def jac_post(eta,y):
    a, gamma, rho_m, rho_r, delta_m, delta_r, theta_r, epsilon = y
    rho_tot= rho_m + rho_r
    #M_Pl = 1.0

    J = np.zeros((8,8))
    J[0,0]= 2*a*np.sqrt(rho_tot/(3*M_Pl**2)) # da'/da
    J[0,1]= 0 # da'/dgamma
    J[0,2]= a**2/(2*np.sqrt(3*rho_tot*M_Pl**2)) # da'/drho_m
    J[0,3]= a**2/(2*np.sqrt(3*rho_tot*M_Pl**2)) # da'/drho_r
    J[0,4]= 0 # da'/ddelta_m
    J[0,5]= 0 # da'/ddelta_r
    J[0,6]= 0 # da'/dtheta_r
    J[0,7]= 0 # da'/depsilon
    J[1,0]= - (2*np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) + np.sqrt(3)*(rho_m*delta_m+rho_r*delta_r)/(M_Pl*np.sqrt(rho_tot)) #dgamma'/da'
    J[1,1]= 0 # dgamma'/dgamma
    J[1,2]= - (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a*rho_tot**(3/2)) + a*np.sqrt(3)*(delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*M_Pl*(rho_tot)**(3/2))
    J[1,3]= - (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a*rho_tot**(3/2)) + a*np.sqrt(3)*(delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*M_Pl*(rho_tot)**(3/2))
    J[1,4]= a*np.sqrt(3/rho_tot)*rho_m/M_Pl # dgamma'/ddelta_m
    J[1,5]= a*np.sqrt(3/rho_tot)*rho_r/M_Pl # dgamma'/ddelta_r
    J[1,6]= 0 # dgamma'/theta_r
    J[1,7]= ((2*k**2)/a)*np.sqrt((3*M_Pl**2)/rho_tot) # dgamma'/depsilon
    J[2,0]= -3*rho_m*np.sqrt(rho_tot/(3*M_Pl**2)) # drho_m'/da
    J[2,1]= 0 # drho_m'/ dgamma
    J[2,2]= -a*np.sqrt(3*rho_tot/M_Pl**2) - (3*a*rho_m)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # drho_m'/drho_m
    J[2,3]= -(3*a*rho_m)/(2*np.sqrt(3*M_Pl**2*rho_tot)) # drho_m'/drho_r
    J[2,4]= 0 
    J[2,5]= 0
    J[2,6]= 0
    J[2,7]= 0 # drho_m'/depsilon
    J[3,0]= -4*rho_r*np.sqrt(rho_tot/(3*M_Pl**2)) # drho_r'/da
    J[3,1]= 0 # drho_r'/ dgamma
    J[3,2]= - (2*a*rho_r)/(np.sqrt(3*M_Pl**2*rho_tot)) # drho_r'/drho_m 
    J[3,3]= -4*a*np.sqrt(rho_tot/(3*M_Pl**2)) - (2*a*rho_r)/(np.sqrt(3**M_Pl**2*rho_tot)) # drho_r'/drho_r
    J[3,4]= 0
    J[3,5]= 0
    J[3,6]= 0
    J[3,7]= 0
    J[4,0]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) - (np.sqrt(3*M_Pl**2)/(2*np.sqrt(rho_tot)))*(rho_m*delta_m+rho_r*delta_r) # ddelta_m'/da
    J[4,1]= 0 # ddelta_m'/dgamma 
    J[4,2]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) - ((a*np.sqrt(3))/(2*M_Pl))*((delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*rho_tot**(3/2))) # ddelta_m'/drho_m
    J[4,3]= (np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) - ((a*np.sqrt(3))/(2*M_Pl))*((delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*rho_tot**(3/2)))# ddelta_m'/drho_r
    J[4,4]= - a/(2*M_Pl)*np.sqrt(3/rho_tot)*rho_m # ddelta_m'/ddelta_m
    J[4,5]= - a/(2*M_Pl)*np.sqrt(3/rho_tot)*rho_r # ddelta_m'/ddelta_r
    J[4,6]= 0
    J[4,7]= -k**2/a*np.sqrt((3*M_Pl**2)/rho_tot) # ddelta_m'/depsilon
    J[5,0]= 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(a**2*np.sqrt(rho_tot)) - 4/3*np.sqrt(3)/(2*M_Pl*np.sqrt(rho_tot))*(rho_m*delta_m+rho_r*delta_r) # ddelta_r'/da
    J[5,1]= 0 # ddelta_m'/dgamma
    J[5,2]= - 4/3*((a*np.sqrt(3))/(2*M_Pl))*((delta_m*(2*rho_r+rho_m)-rho_r*delta_r)/(2*rho_tot**(3/2))) + 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) # ddelta_r'/drho_m
    J[5,3]= - 4/3*((a*np.sqrt(3))/(2*M_Pl))*((delta_r*(2*rho_m+rho_r)-rho_m*delta_m)/(2*rho_tot**(3/2))) + 4/3*(np.sqrt(3*M_Pl**2)*k**2*epsilon)/(2*a*rho_tot**(3/2)) # ddelta_r'/drho_r
    J[5,4]= - (2*a*rho_m)/(M_Pl*np.sqrt(3*rho_tot)) # ddelta_r'/ddelta_m
    J[5,5]= - (2*a*rho_r)/(M_Pl*np.sqrt(3*rho_tot)) # ddelta_m'/ddelta_r
    J[5,6]= -4/3 # ddelta_m'/dtheta_r
    J[5,7]= -4/3*(k**2/a)*np.sqrt((3*M_Pl**2)/rho_tot) # ddelta_m'/depsilon
    J[6,0]= 0 # dtheta_r'/da
    J[6,1]= 0 # dtheta_r'/dgamma
    J[6,2]= 0 # dtheta_r'/drho_m
    J[6,3]= 0 # dtheta_r'/drho_r
    J[6,4]= 0 # dtheta_r'/ddelta_m
    J[6,5]= k**2/4 # dtheta_r'/ddelta_r
    J[6,6]= 0 # dtheta_r'/dtheta_r
    J[6,7]= 0 # dtheta_r'/depsilon
    J[7,0]= (4*a*rho_r*theta_r)/(3*k**2) # depsilon'/da
    J[7,1]= 0 # depsilon'/dgamma
    J[7,2]= 0 # depsilon'/drho_m
    J[7,3]= (2*a**2*theta_r)/(3*k**2) # depsilon'/drho_r
    J[7,4]= 0 # depsilon'/ddelta_m
    J[7,5]= 0 # depsilon'/ddelta_r
    J[7,6]= (2*a**2*rho_r)/(3*k**2) # depsilon'/dtheta_r 
    J[7,7]= 0 # depsilon'/depsilon
    
    return J

## Initial Conditions

In [17]:
# Physical constants and parameters
# We assume natural units c=\bar{h}=1 and given the redefinition of the quantities necessary to use the following formulas for the
# initial conditions, we are free to choose the units that we want, since H, rho_m, rho_r and GammaT are all dimensionless.
M_Pl=1
M_PBH_i = 1e4  # Initial PBH mass in grams
M_Pl_to_g= 4.340*10**(-6) # 1 MPl in grams
M_PBH_to_GeV= 5.62*10**23 # 1 gram express in GeV
M_Pl_to_GeV= 2.44*10**18 # reduced Planck mass in GeV

# Compute evaporation time from the initial mass of the PBHs (monochromatic mass distribution)
A = np.pi * 3.8 * 108 * M_Pl**4 / 480 
teva = (M_PBH_i)**3 / (3 * A)
#teva = (M_PBH_i/M_Pl_to_g)**3 / (3 * A)
beta_min= 6.5*10**(-10)*(M_PBH_i/10**4)**(-1)

# Initial conditions for background evolution before t_eva
beta_i = 2.1725e-3 #2.18e-3 value for 10^4 g, 1.93e-1 for ratio=1000 in GeV, 2.9e-2 for ratio=225 in GeV
sqrt_beta = np.sqrt(1 + beta_i)
denominator = (-1 + beta_i + sqrt_beta)**2
rho_r0 = (27/4) * (beta_i**2) / denominator
rho_m0 = beta_i * rho_r0
a0 = 1.0
#M_PBH0= M_PBH_i
GammaT0 = a0/(3 * teva)
#GammaT0= A/(M_PBH_i*M_PBH_to_GeV)**3
#GammaT0= A/(M_PBH_i/M_Pl_to_g)**3


# Perturbation initial conditions before t_eva
C0 = 5/6
eta0 = 10**(-15)
def delta_r0(k):
    res = - (2/3) * C0 * (k * eta0)**2
    return res
def delta_m0(k):
    res = (3/4) * ( - (2/3) * C0 * (k * eta0)**2 ) # delta_m0 = 3/4 * delta_r0
    return res
theta_m0 = 0.0
def theta_r0(k):
    res = - (1/18) * C0 * (k**4 * eta0**3)
    return res
def gamma0(k):
    res = C0 * (k * eta0)**2
    return res
def epsilon0(k):
    res = 2 * C0 - (1/18) * C0 * (k * eta0)**2
    return res

## Solving once the background to find eta_eq2, test the ratio eta_eq2/eta_eq1 , and eta_eva

In [18]:
# Pack initial conditions for background
y0_background = [rho_m0, rho_r0, a0, GammaT0]
print(y0_background)

# Solve background ODE
eta_max = 1e7
sol_background = solve_ivp(background_odes, [0, eta_max], y0_background, max_step=eta_max/10**6, events=stop_event, method='BDF',
                           rtol=1e-7, atol=1e-15)

#print("solve_ivp status after numerical integration= "+str(sol_background.status))
#print("solve_ivp message after numerical integration= "+str(sol_background.message))
#print("Number of evaluations of the right-hand side= "+str(sol_background.nfev))

# Extract background solution and save them in an array (eta, rho_m, rho_r, a, GammaT)
ysol_back=np.array([sol_background.t,sol_background.y[0], sol_background.y[1], sol_background.y[2], sol_background.y[3]])
rho_tot_bg = ysol_back[1] + ysol_back[2]
H_bg = ysol_back[3] * np.sqrt(rho_tot_bg / (3 * M_Pl**2))

#print(shape(ysol_back))
#print("Time", ysol_back[0,0], ysol_back[0,-1])
#print("GammaT", ysol_back[4,0], ysol_back[4,-1])
#print(r"Final values of $\rho_m$ and $\rho_r$ over $\rho_{tot}$", ysol_back[1,-1]/rho_tot_bg[-1], ysol_back[2,-1]/rho_tot_bg[-1])

# Interpolate background variables
rho_m_interp = interp1d(ysol_back[0,:], ysol_back[1,:], kind='cubic', fill_value="extrapolate")
rho_r_interp = interp1d(ysol_back[0,:], ysol_back[2,:], kind='cubic', fill_value="extrapolate")
a_interp = interp1d(ysol_back[0,:], ysol_back[3,:], kind='cubic', fill_value="extrapolate")

# Find eta_eq2: solve rho_m - rho_r = 0 for second crossing
diff = ysol_back[1,:] - ysol_back[2,:]
z = np.sign(diff)
zeros = []
for i in range(len(z)-1):
    if z[i]*z[i+1]<0:
        r = root_scalar(lambda t: rho_m_interp(t) - rho_r_interp(t), bracket=[ysol_back[0,i], ysol_back[0,i+1]])
        zeros.append(r.root)
if len(zeros)==0:
    raise ValueError("Insufficient crossings found.")
else:
    eta_eq1=zeros[0]
    #print(eta_eq1)
    eta_eq2 = zeros[1]

eta_eva = ysol_back[0,-1]
a_eq1 = a_interp(eta_eq1)
print(a_eq1)
print("eta_eq1= "+str(eta_eq1), "eta_eq2= "+str(eta_eq2), "eta_eq2 / eta_eq1= "+str(eta_eq2/eta_eq1), "eta_eva= "+str(eta_eva))

#fig1=plt.figure(1)
#plt.plot(ysol_back[0,:]/eta_eq2, ysol_back[4,:], linewidth = 1, color="blue", label=r"$\tilde{\Gamma}$")
#grid(True); plt.legend(fontsize=15);
#xlabel(r'$\eta/\eta_{eq,2}$', fontsize=15);
#yscale('log');
#xscale("log");
#plt.xlim(0, 1.1e0)
#plt.gca().xaxis.set_major_formatter(ScalarFormatter())
#plt.gca().yaxis.set_major_formatter(ScalarFormatter())
#plt.show()

[np.float64(0.006519857957801541), np.float64(3.0010853660766585), 1.0, 2.686061718819273e-12]
460.31445629533596
tau_eq1= 380.2638252636021 tau_eq2= 85561.53618201212 tau_eq2 / tau_eq1= 225.00572102197768 tau_eva= 87813.38060809873


/tmp/ipykernel_172124/2698865198.py:17: RuntimeWarning: invalid value encountered in sqrt
  da = a**2 * np.sqrt((rho_m + rho_r)/(3*M_Pl**2))


## Calculate $k_{NL}$ using Inomata's estimate

In [6]:
"""
We now compute the cut-off scale that will enter in the derivation of P_GW
This scale is global, it doesn't depend on the specific gravitational potential derived numerically,
and it enters in the eRD, so that by the end of MD $\\delta_m$ will be larger than one. Smaller k, i.e. larger $\\lambda$, will enter
in the horizon during the MD and will behave correctly. 
"""

# Define the residual to find k_cut given by eq. C3 in Inomata's paper
def residual_k_cut(k_cut):
    x = k_cut * eta_eq1
    phi_pl = (np.log(1+0.146*x)/(0.146*x) ) * ( 1+0.242*x + (1.01*x)**2 + (0.341*x)**3 + (0.418*x)**4 )**(-0.25)
    # left‐hand side of (C3):
    LHS = k_cut * np.sqrt(phi_pl) * ((2.1*10**(-9))**0.25)
    # right‐hand side of (C3):
    RHS = 2 * np.sqrt(5.0 / 2.0) / eta_eq2
    return LHS - RHS


# Solve for k_NL using a root‐finder:
k_min= 1e-6
k_max= 1e15
sol = root_scalar(
    residual_k_cut,
    bracket=[k_min, k_max],
    method='brentq',    # or 'brentq'
    x0= 800,
    xtol=1e-12,
)

if not sol.converged:
    raise RuntimeError("Root‐finding did not converge.")

# Find k_cut = k_NL
k_NL = sol.root
print(f"k_NL = {k_NL:.3e},", f"k_NL*eta_eq2 = {k_NL*eta_eq2}")

k_NL = 8.663e-03, k_NL*tau_eq2 = 740.3744519947294


# Computing $\Omega_{GW}$ using Inomata's instructions (RD and $x_{eva}$) and applying oscillation average

In [7]:
# Scale-invariant primordial power spectrum used in all integrals and computations
def Pz(k,k_cut):
    amp = 2.1e-9
    ns = 1.0
    kcmb = 0.05
    if k < k_cut:
        res = amp
    else:
        res = 0
    return res

def phi_plateau(x):
    func = np.log(1+0.146*x)/(0.146*x)*(  1+0.242*x + (1.01*x)**2 + (0.341*x)**3 + (0.418*x)*4)*(-0.25)
    return func

def J(x, x0):
    arg = (x-x0/2) / np.sqrt(3)
    sphbessel =  np.sin(arg) / arg**2 - np.cos(arg) / arg  # shperical Bessel function j_1
    func = (3 / arg ) * sphbessel
    return func

def dJ(x, x0):
    arg = (x-x0/2) / np.sqrt(3)
    sphbessel =  np.sin(arg) / arg**2 - np.cos(arg) / arg  # shperical Bessel function j_1
    dervsphbessel =  2 * np.cos(arg) / arg**2 + np.sin(arg) * ( 1/arg - 2/ arg**3 )  # derivative of shperical Bessel function j_1
    func = (np.sqrt(3) / arg) * (dervsphbessel -  sphbessel / arg)
    return func

def Y(x, x0):
    arg = (x-x0/2) / np.sqrt(3)
    sphbessel = -np.cos(arg) / arg**2 - np.sin(arg) / arg   # shperical Bessel function y_1
    func = (3 / arg ) * sphbessel
    return func

def dY(x, x0):
    arg = (x-x0/2) / np.sqrt(3)
    sphbessel =  - np.cos(arg) / arg**2 - np.sin(arg) / arg  # shperical Bessel function y_1
    dervsphbessel = 2 * np.sin(arg) / arg**2 + np.cos(arg) * ( 2/arg**3 - 1/ arg )  # derivative of shperical Bessel function y_1
    func = (np.sqrt(3) / arg) * (dervsphbessel -  sphbessel / arg)
    return func

def matching_coeffs(x0):
    A = dY(x0, x0) / (dY(x0, x0) * J(x0, x0) - dJ(x0, x0) * Y(x0, x0))
    B = - dJ0 / (dY(x0, x0) * J(x0, x0) - dJ(x0, x0) * Y(x0, x0))
    return A, B

# Define the gravitational potential Phi_norm  and its derivative that enters in the source function
def Phi_oscfit (x, k, eta_eva, eta_eq2, best_S):
    x0 = k * eta_eva
    #x_vals= k*eta_tot_450 # I would use the time array if I want to use numerical data, which right now I'm not doing
    if k<1/eta_eq2:
        A, B = matching_coeffs(x0)
        func = A*J(x, x0) + B*Y(x, x0)
        return func
        
    elif k>3000/eta_eq2:
        S_low = ( np.sqrt(6) / ( k * eta_eva ) )**(1/3) * phi_plateau(k*eta_eq1)
        A, B = matching_coeffs(x0)
        func = A*J(x, x0) + B*Y(x, x0)
        return S_low * func
        
    else:
        A, B = matching_coeffs(x0)
        func = A*J(x, x0) + B*Y(x, x0)
        return best_S(k * eta_eq2) * func 

def dPhi_oscfit (x, k, eta_eva, eta_eq2, best_S):
    x0 = k * eta_eva
    #dphi_norm = np.gradient(Phi_norm, x) # This would work if Phi_norm was an array of values, not an analytical function
    if k < 1 / eta_eq2:
        A, B = matching_coeffs(x0)
        func = A * dJ(x, x0) + B * dY(x, x0)
        return func
        
    elif k > 3000 / eta_eq2:
        S_low = (np.sqrt(6)/(k * eta_eva ))**(1/3) * phi_plateau(k * eta_eq1)
        A, B = matching_coeffs(x0)
        func = A * dJ(x, x0) + B * dY(x, x0)
        return S_low * func
        
    else:
        A, B = matching_coeffs(x0)
        func = A*dJ(x, x0) + B*dY(x, x0)
        return best_S(k * eta_eq2) * func 

# Define the scale factor a valid for RD epoch
def a_RD(eta, eta_eva, eta_eq1, a_eq1):
    eta_star = eta_eq1 / ( np.sqrt(2) - 1)
    res = a_eq1 * ( 2 * eta * (eta_eva + eta_star) -eta_eva**2 ) / eta_star**2
    return res

# Define the Hubble parameter H valid for the RD epoch that enter in the source term f
def H_RD (eta, eta_eva, eta_eq1):
    eta_star = eta_eq1 / (np.sqrt(2)-1)
    res = 1 / (eta - eta_eva**2 / (2 * (eta_eva + eta_star)) )
    return res


# Define the source term f
def f_source(v, u, xbar, eta_eva, k, best_S): # here xbar is the variable that is integrated over in I_uv
    pre = 9.0 / 100.0 # prefactor obtained assumin w=1/3 for RD
    # arguments:
    ux = u * xbar
    uk = u * k
    ueta_eva = u * eta_eva
    vx = v * xbar
    vk = v * k
    veta_eva = v * eta_eva
    ux0 = u * best_x0
    vx0 = v * best_x0
    # evaluate Φ_norm at (u xbar), (v xbar):
    Phi_u = Phi_oscfit(ux, uk, ueta_eva, best_S)
    Phi_v = Phi_oscfit(vx, vk, veta_eva, best_S)
    # derivatives:
    dPhi_u = dPhi_oscfit(ux, uk, ueta_eva, best_S)
    dPhi_v = dPhi_oscfit(ux, uk, ueta_eva, best_S)
    # Hubble at xbar:
    term1 = 12.0 * (Phi_u * Phi_v)
    term2 = 4.0 * H_RD**(-1) * k* ( u * dPhi_u * Phi_v + v * Phi_u * dPhi_v ) 
    term3 = 4.0 * H_RD**(-2) * k**2 * u * v * ( dPhi_u * dPhi_v )
    return pre * (term1 + term2 + term3)


"""
Revised up to here
"""


# Define and compute the integral Is
def I_s(v, u, xbar, x_eva, x_fin, k, num, best_S):
    pre_factor= 2 * xbar / x_eva -1
    fvals= f_source(v, u, xbar, x_eva, k, num, best_S)
    if x_fin==x_eva:
        return 0
    else:
        return simpson(pre_factor*np.sin(xbar)*fvals, xbar)

# Define and compute the integral Ic
def I_c(v, u, xbar, x_eva, x_fin, k, num, best_S):
    pre_factor= 2 * xbar / x_eva -1
    fvals= f_source(v, u, xbar, x_eva, k, num, best_S)
    if x_fin==x_eva:
        return 0
    else:
        return simpson(pre_factor*np.cos(xbar)*fvals, xbar)

# Define and compute the integral I that contains all the time dependencies
def I_uv(v, u, xbar, x_eva, x_fin, k, num, best_S):
    funs = I_s(v, u, xbar, x_eva, x_fin, k, num, best_S)
    func = I_c(v, u, xbar, x_eva, x_fin, k, num, best_S)
    pre_factor= 2 * x / x_eva -1
    return (funs*2.0+func*2.0)/(2*pre_factor**2.0)

# Geometrical prefactor that enters in the P_h integral
def F_geom(u, v):
    num = 4.0*v*v - (1.0 + v*v - u*u)**2
    return (num**2) / (16.0 * u*u * v*v)

In [8]:
# Compute the time-averaged power spectrum of GWs
def compute_P_h(k, best_x0, x_factor, k_cut, num, best_S, Nx=10*2, Nu=10*5, Nv=10**5):
    x_fin = x_factor * best_x0 # while theoretically the time integral goes to infinity, we choose a time when the integrals have converged sufficiently
    xbar_array= np.linspace(best_x0, x_fin, Nx) # array of x_values that are integrated over in the I_uv integral
    v_max = k_cut/k # this corresponds to the maximum value of v such that v_max*k=k_NL in the Power Spectrum
    #print(v_max)
    v_array=np.linspace(1e-3, v_max, Nv) # array of v_values
    P_h_v = np.zeros_like(v_array) # Array that contains the value of P_h for each v value (u has been integrated)
    
    for iv, vv in enumerate(v_array): # for each value of v, we create an array of values of u
        u_min = abs(1.0 - vv)
        u_max = 1.0 + vv
        u_array = np.linspace(u_min, u_max, Nu)
        P_h_u = np.zeros_like(u_array) # Array that contains the value of the P_h integrand for each u and v value
        
        for iu, uu in enumerate(u_array):
            if uu*k > k_cut:
                P_h_u[iu] = 0.0
                continue
            
            Ival = I_uv(uu, vv, xbar_array, best_x0, x_fin, k, num, best_S)
            P_h_u[iu]= F_geom(uu, vv) * Ival * Pz(k*uu, k_cut) * Pz(k*vv, k_cut)
            P_h_v[iv] = simpson(P_h_u, u_array)
        return 4.0 * simpson(P_h_v, v_array)

# Computes the density parameters of GWs
def Omega_GW(k, best_x0, x_factor, k_cut, num, best_S, **grid_kwargs):
    x_eva = best_x0
    x_fin = x_factor * x_eva
    
    # background prefactor at x_fin
    H_fin = H_RD(x_fin, x_eva, k)
    pref = (1.0/24.0) * (k / H_fin)**2
    P_h = compute_P_h(k, x_eva, x_factor, k_cut, num, best_S, **grid_kwargs)
    return pref * P_h

In [13]:
# WE MOVE TO THE COMPUTATION OF THE GRAVITATIONAL WAVES, STARTING BY CONSIDERING JUST ONE k, AND WE START BY DOING EXACTLY LIKE
# INOMATA, USING THE ANALYTICAL POTENTIAL AND x_eva FOR ALL THE k

best_S=np.load("First-order_Perturbations/data/Sk/best_S.npy")
array1=np.arange(1e0, 1e1, 1)
array2=np.arange(1e1, 1e2, 2)
array3=np.arange(1e2, 3.01e3, 10)
num=np.concatenate((array1, array2, array3))

# Interpolate best_S to have a continuous function for all effective momenta in the integrals
S_func= interp1d(num, best_S, kind='cubic', fill_value="extrapolate")

# Scale for which we compute Omega_GW. For every k<k_NL, I compute Phi_num and determine the best_S and best_x0 to fit Phi_num with
# the analytical formula Phi_fit that describes the gravitational potential during RD. Being this Phi_fit that enters in the
# computation of Omega_GW, all the integrals are analytical, no numerical xdata and ydata/Phi_num is needed.

# Let's now compute Omega_GW for all k
num_array1=np.linspace(1e-3, 1e0, 10**3)
num_array2=np.linspace(1e0, 1e4, 10**4)
num_array=np.concatenate((num_array1, num_array2[1:]))
Omegak_array=np.zeros_like(num_array)
for i in range(len(num_array)):
    k_mode=num_array[i]/eta_eq2
    best_x0=k_mode*ysol_back[0,-1]
    x_factor=3.5
    #time_i= time.time()
    Omegak_array[i]=Omega_GW(k_mode, best_x0, x_factor, k_NL, num_array[i], S_func, Nx=100, Nu=100, Nv=100)
    #time_f= time.time()
    #print(f"Done k_num={num_array[i]:.3f}, Omega={Omegak_array[i]:.2e}")
    #print("Elapsed time to compute one Omega_k with Nx=Nu=Nv=100 is", time_f-time_i)

fig1=plt.figure(1)
plt.plot(num_array, Omegak_array, color="orange", linewidth=1.5, label=r"$\eta_{eq,2}/\eta_{eq,1}=225$")
# Add labels and a title
plt.xlabel(r'k*$\eta_{eq,2}$')
plt.ylabel(r'$\Omega_{GW}/A_s^2$')
xscale("log")
yscale("log")
plt.grid(True)
plt.show()

/tmp/ipykernel_5467/1854419552.py:126: DeprecationWarning: You are passing x=[0.00102632 0.00105223 0.00107815 0.00110407 0.00112999 0.0011559
 0.00118182 0.00120774 0.00123365 0.00125957 0.00128549 0.00131141
 0.00133732 0.00136324 0.00138916 0.00141507 0.00144099 0.00146691
 0.00149283 0.00151874 0.00154466 0.00157058 0.00159649 0.00162241
 0.00164833 0.00167425 0.00170016 0.00172608 0.001752   0.00177791
 0.00180383 0.00182975 0.00185567 0.00188158 0.0019075  0.00193342
 0.00195933 0.00198525 0.00201117 0.00203709 0.002063   0.00208892
 0.00211484 0.00214075 0.00216667 0.00219259 0.0022185  0.00224442
 0.00227034 0.00229626 0.00232217 0.00234809 0.00237401 0.00239992
 0.00242584 0.00245176 0.00247768 0.00250359 0.00252951 0.00255543
 0.00258134 0.00260726 0.00263318 0.0026591  0.00268501 0.00271093
 0.00273685 0.00276276 0.00278868 0.0028146  0.00284052 0.00286643
 0.00289235 0.00291827 0.00294418 0.0029701  0.00299602 0.00302194
 0.00304785 0.00307377 0.00309969 0.0031256  0.003151

KeyboardInterrupt: 